In [2]:
# In this notebook are the codes for the analytic method used to compare with the small_box algorithm
# See sub-group slides by Mark Fornace at the beginning of summer ()
# Current sequence and initial conditions from Othmer thesis (figure 3.3)

In [2]:
import math
import numpy as np
import nupack4
from nupack4 import State, Model
import scipy.stats
from scipy.sparse.linalg import spsolve

In [5]:
# This notebook contains the analytical analysis steps for a given sequence.

In [3]:
# 1)
# Finding the rate matrix with enumerated
#all_states, small_pi, R = State('GUCGCGUCGCGUCGCUAUGCGAC',model=Model(T=298, gt = True), kind = 'slow').rate_matrix(True)
mod = Model(dangles='none', gt=True)
all_states, small_pi, R = State('GCGUCGCGUCGCUAUGC',model=mod, kind = 'slow').rate_matrix(True)



In [4]:
# 2)
# Z matrix (fundamental matrix), see Fornace report and white board on the 27.06.2017

#all_states = sorted(list(State('GUCGCGUCGCGUCGCUAUGCGAC',model=Model(T=298, gt = True), kind = 'slow').enumerated()))

#small_pi = np.asarray([w.boltz for w in all_states])
#small_pi /= small_pi.sum()
#np.shape(small_pi)

big_pi = np.zeros([len(small_pi),len(small_pi)]) + small_pi
#for i in range(len(small_pi)):
#    for j in range(len(small_pi)):
#        big_pi[i][j] = small_pi[j]


Z = np.linalg.inv(R+big_pi) - big_pi



In [5]:
# 3) Find the matrix S
small_pi_tilde_new
#S = Z/small_pi



NameError: name 'small_pi_tilde_new' is not defined

In [7]:
# 4) Take Sxx as the elements of S on the BC

# Find positions of macro state MFE's in matrix
#macro_States = ['..........(((((...)))))', '(((((((.((...)).)))))))', '.....((((((......))))))','..(((...)))((((...)))).','...((...))(((((...)))))']
macro_States = ['.....((((....))))','((((.((...)).))))','(((......))).....']

location = []
for i in range(len(macro_States)):
    for j in range(len(all_states)):
        if macro_States[i] == all_states[j].dp:
            location.append(j)
print(location)
#Sxx = np.zeros([len(macro_States),len(macro_States)])

#for i,x in enumerate(location):
#    for j,y in enumerate(location):
#        Sxx[i][j] = S[x][y]

#print(Sxx)

[51, 773, 495]


In [8]:
# 5) Calculate S_tilda as seen on table 11.07.17

macro_small_pi = small_pi[location]
macro_small_pi /= macro_small_pi.sum()

S_tilde = (np.identity(len(macro_States))-np.outer(macro_small_pi,macro_small_pi)/(np.linalg.norm(macro_small_pi))**2 ) @ Sxx @ (np.identity(len(macro_States)) - np.outer(macro_small_pi,macro_small_pi)/(np.linalg.norm(macro_small_pi))**2 )

NameError: name 'Sxx' is not defined

In [11]:
# 6) Calculate small_pi_tilde as 0 eigenvector of S_tilde
x,y = np.linalg.eig(S_tilde)
small_pi_tilde = y[np.argmin(abs(x))]

NameError: name 'S_tilde' is not defined

In [10]:
# 7) Calculate Z_tilde 
Z_tilde = S_tilde * small_pi_tilde

NameError: name 'S_tilde' is not defined

In [9]:
# 8) Find R_tilde

big_pi_tilde = np.zeros([len(macro_small_pi),len(macro_small_pi)])
for i in range(len(macro_small_pi)):
    for j in range(len(macro_small_pi)):
        big_pi_tilde[i][j] = macro_small_pi[j]


R_tilde = np.linalg.inv(Z_tilde-big_pi_tilde) + big_pi_tilde



NameError: name 'Z_tilde' is not defined

In [ ]:
# Use the rate matrix found in 8) to solve the ODE and draw over a certain timescale.

import matplotlib.pyplot as plt
import seaborn as sns

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
          '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
          '#bcbd22', '#17becf']
sns.set(style='whitegrid', palette=colors, rc={'axes.labelsize': 16})

# The following is specific Jupyter notebooks
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}


def nytt(vec,t):
    dvecDt = vec@R_tilde
    return dvecDt

y0 = np.array([1,0])
t = np.linspace(0,10e-4,num = 1000)

vec = scipy.integrate.odeint(nytt ,y0 ,t)

# Smellir upp grunn
fig, ax = plt.subplots(1, 1)
# Setur merkingar a asa
ax.set_xlabel(r'$Time \ ( s )$')
ax.set_ylabel(r'$Dilution \ (nM)$');

_ = ax.plot(t, np.transpose(vec[:,0])/np.transpose(vec[:,0]+np.transpose(vec[:,1])+np.transpose(vec[:,2])))
_ = ax.plot(t, np.transpose(vec[:,1])/np.transpose(vec[:,0]+np.transpose(vec[:,1])+np.transpose(vec[:,2])))
_ = ax.plot(t, np.transpose(vec[:,2])/np.transpose(vec[:,0]+np.transpose(vec[:,1])+np.transpose(vec[:,2])))

ax.legend(('Macrostate 1', 'Macrostate 2', 'Macrostate 3'), loc='upper right');





In [13]:
#####################################################################################################################
A = tuple(location)
B = tuple(i for i in range(len(small_pi)) if i not in A)

M = R[:, B][B, :]
rhs = -R[:, A][B, :]

CB = spsolve(M, rhs)



In [14]:
C = np.zeros([len(small_pi), len(A)])
C[A, :] = np.eye(len(A))

print(CB.shape)
print(C[B, :].shape)

C[B, :] = CB.todense()

(776, 3)
(776, 3)


In [16]:
# Try different method to find S_tilde (instead of using small_pi of the MFEs you use the one written on table 19.07)

#SxA = np.zeros([len(macro_States),len(S[0])])

#for i,x in enumerate(location):
#        SxA[i][:] = S[x][:]
#SxA = SxA.transpose()



#C = ((1 - SxA@(np.linalg.inv(Sxx))@(np.ones([len(macro_States),len(macro_States)])))/((np.linalg.inv(Sxx.transpose())).sum()) + SxA)@np.linalg.inv(Sxx.T)

small_pi_tilde_new = small_pi@C

R_tilde_new = np.diag(1/(small_pi_tilde_new))@(C.T)@(np.diag(small_pi))@R@C

In [90]:
# Use the rate matrix found in 8) to solve the ODE and draw over a certain timescale.
# The following is for the small Othmer case, time-scale appears to be very different though and 
# curves are different as well.
import matplotlib.pyplot as plt
import seaborn as sns

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
          '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
          '#bcbd22', '#17becf']
sns.set(style='whitegrid', palette=colors, rc={'axes.labelsize': 16})

# The following is specific Jupyter notebooks
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

print(R_tilde_new.sum(1))

def nytt(vec,t):
    dvecDt = vec@R_tilde_new
    return dvecDt

y0 = np.array([1,0,0,0,0])
t = np.linspace(0,1e+2,num = 1000000)

vec = scipy.integrate.odeint(nytt ,y0 ,t)

# Smellir upp grunn
fig, ax = plt.subplots(1, 1)
# Setur merkingar a asa
ax.set_xlabel(r'$Time \ ( s )$')
ax.set_ylabel(r'$Dilution \ (nM)$');
ax.set_xscale('log')
ax.set_xlim(1e-7, 1e8)


_ = ax.plot(t/1.e-6, np.transpose(vec[:,0])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])+np.transpose(vec[:,3])+np.transpose(vec[:,4])))
_ = ax.plot(t/1.e-6, np.transpose(vec[:,1])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])+np.transpose(vec[:,3])+np.transpose(vec[:,4])))
_ = ax.plot(t/1.e-6, np.transpose(vec[:,2])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])+np.transpose(vec[:,3])+np.transpose(vec[:,4])))
_ = ax.plot(t/1.e-6, np.transpose(vec[:,3])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])+np.transpose(vec[:,3])+np.transpose(vec[:,4])))
_ = ax.plot(t/1.e-6, np.transpose(vec[:,4])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])+np.transpose(vec[:,3])+np.transpose(vec[:,4])))


print(np.shape(vec))

ax.legend(('Macrostate 1', 'Macrostate 2', 'Macrostate 3', 'Macrostate 4', 'Macrostate 5'), loc='upper left');
#plt.savefig('DiffEqSolved_AnalyticRateMatrix_SYSTEM2.png', bbox_inches='tight')

[ -4.43833414e-10   5.78533014e-11   1.19552572e-10   2.32830644e-10
   0.00000000e+00]
(1000000, 5)


In [70]:
macro_States[0]

'..........(((((...)))))'

In [ ]:
all_states = sorted(list(State('GUCGCGUCGCGUCGCUAUGCGAC',model=Model(T=298, gt = True), kind = 'slow').enumerated()))

In [ ]:
#print(max(all_states, key = lambda w:w.energy))
for i in macro_States:
    if any(x.dp == i for x in all_states):
        print('works')
        print()
    else:
        print('is not there')

In [ ]:
w=State('GUCGCGUCGCGUCGCUAUGCGAC', '(((((((.((...)).)))))))', model=Model(T=298, gt = True), kind = 'slow')


[i for i in State('GUCGCGUCGCGUCGCUAUGCGAC',model=Model(T=298, gt = True), kind = 'slow').enumerated() if i.dp == '(((((((.((...)).)))))))']


In [ ]:
Model(T=298, gt = True).gt_allowed

In [ ]:
states.p.rate_matrix

In [ ]:
print(Z)
print(big_pi@R)

In [89]:
w = State('AAAAT+ATTTT', kind='slow', model=Model(dangles='none'))
print(w.complexes.join_rates)

[
        0        0        0   2.0745
        0        0        0        0
        0        0        0        0
   0.5186        0        0        0


        0        0        0   0.5186
        0        0        0        0
        0        0        0        0
   2.0745        0        0        0
]


In [27]:
# Use the rate matrix found in 8) to solve the ODE and draw over a certain timescale.
# Wolfe case, 17nt case used for new algorithm
import matplotlib.pyplot as plt
import seaborn as sns

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
          '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
          '#bcbd22', '#17becf']
sns.set(style='whitegrid', palette=colors, rc={'axes.labelsize': 16})

# The following is specific Jupyter notebooks
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

print(R_tilde_new.sum(1))

def nytt(vec,t):
    dvecDt = vec@R_tilde_new
    return dvecDt

y0 = np.array([1,0,0])
t = np.linspace(0,0.001,num = 1000)

vec = scipy.integrate.odeint(nytt ,y0 ,t)

# Smellir upp grunn
fig, ax = plt.subplots(1, 1)
# Setur merkingar a asa
ax.set_xlabel(r'$Time \ ( s )$')
ax.set_ylabel(r'$Probability$');
#ax.set_xscale('log')
#ax.set_xlim(1e-7, 1e8)


_ = ax.plot(t/1.e-6, np.transpose(vec[:,0])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])))
_ = ax.plot(t/1.e-6, np.transpose(vec[:,1])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])))
_ = ax.plot(t/1.e-6, np.transpose(vec[:,2])/(np.transpose(vec[:,0])+np.transpose(vec[:,1])+np.transpose(vec[:,2])))


print(np.shape(vec))

ax.legend(('Macrostate 1', 'Macrostate 2', 'Macrostate 3'), loc='center right');
plt.savefig('DiffEqSolved_AnalytdicRateMatrix_WolfeCase.png', bbox_inches='tight')

[ -8.27640179e-11  -8.75672868e-11   1.99179340e-10]
(1000, 3)
